In [143]:
#!g2.mig
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

2023-01-18 13:52:23.249418: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 13:52:26.431518: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 2629 MB memory:  -> device: 0, name: GRID A100X-1-5C MIG 1g.5gb, pci bus id: 0000:8c:00.0, compute capability: 8.0


Found GPU at: /device:GPU:0


In [144]:
#!g2.mig
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GRID A100X-1-5C MIG 1g.5gb


In [12]:
#!g2.mig
%pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.1 MB 1.8 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [14]:
#!g2.mig
%pip install -U numpy

Defaulting to user installation because normal site-packages is not writeable


In [19]:
#!g2.mig
%pip install numpy==1.21

Defaulting to user installation because normal site-packages is not writeable


In [145]:

import warnings
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 2000

In [146]:
%pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("nickmuchi/finbert-tone-finetuned-finance-topic-classification")

model = BertForSequenceClassification.from_pretrained("nickmuchi/finbert-tone-finetuned-finance-topic-classification")

Defaulting to user installation because normal site-packages is not writeable








In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [147]:
import pandas as pd 
df=pd.read_csv('general_dataset.csv')

In [148]:
import numpy as np
df.fillna({'Ticker':'UBER', 'Data source':'Twitter'}, inplace=True)

In [152]:
counter = 0
for i in range(df.shape[0]):
    if type(df.loc[i, 'Headline']) is float:
        counter += 1
        df.loc[i, 'Headline'] = 'No headline'
print(counter)

3


In [153]:
twitter_dataset = df[df['Data source'] == 'Twitter']
twitter_dataset.shape

(870439, 5)

In [154]:
financial_news_dataset = df[df['Data source'] == 'Financial news']
financial_news_dataset.shape

(221513, 5)

In [156]:
sentences = financial_news_dataset.Headline.values
max_len = 0
number = 0
for sent in sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    number += 1
    if number == 10001:
      break

print('Max sentence length: ', max_len)

Max sentence length:  38


In [164]:
predictions = []

In [162]:
def prediction():
  print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
  model.to(device)
    
  model.eval()

  true_labels = []
  number = 0
  # Predict
  for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask = batch

    
    with torch.no_grad():
     
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)

    for logit in outputs.logits:

      logits = logit
      predicted_class_id = logits.argmax().item()
      logits = logits.detach().cpu().numpy()
      predicted_class = model.config.id2label[predicted_class_id]
      predictions.append(predicted_class)
      number += 1

  print('    DONE.')

In [165]:
#!g2.mig
import torch
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader

sentences = financial_news_dataset.Headline.values

input_ids = []
attention_masks = []
batch_number = 0

for sent in sentences:
    
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 38,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt', # Return pytorch tensors.
                        truncation = True
                   )

    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])
    if len(input_ids ) >= 1000:
      batch_number += 1
      input_ids = torch.cat(input_ids, dim=0)
      attention_masks = torch.cat(attention_masks, dim=0)

      batch_size = 32
      prediction_data = TensorDataset(input_ids, attention_masks)
      prediction_sampler = SequentialSampler(prediction_data)
      prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
      print('Batch number', batch_number)
      prediction()
      print('Len of predictions', len(predictions))
      input_ids = []
      attention_masks = []
    else:
        if batch_number >= 221 and len(input_ids) == 513:
            print('Additional batch')
            batch_number += 1
            input_ids = torch.cat(input_ids, dim=0)
            attention_masks = torch.cat(attention_masks, dim=0)
            batch_size = 32
            prediction_data = TensorDataset(input_ids, attention_masks)
            prediction_sampler = SequentialSampler(prediction_data)
            prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
            print('Batch number', batch_number)
            prediction()
            input_ids = []
            attention_masks = []

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2221: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch number 1
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 1000
Batch number 2
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 2000
Batch number 3
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 3000
Batch number 4
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 4000
Batch number 5
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 5000
Batch number 6
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 6000
Batch number 7
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 7000
Batch number 8
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 8000
Batch number 9
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 9000
Batch number 10
Predicting labels for 1,000 test sentences...
    DONE.
Len of predictions 10000
Batch number 11
Predicting labels for 1,000 test

In [166]:
#!g2.mig
print(len(predictions))

221513


In [167]:
#!g2.mig
print(len(sentences))

221513


In [168]:
#!g2.mig
def count_labels(label):
  counter = 0
  for prediction in predictions:
    if prediction == label:
      counter += 1
  return counter


In [169]:
#!g2.mig
print('Analyst Update', count_labels('Analyst Update'))
print('Fed | Central Banks', count_labels('Fed | Central Banks'))
print('Company | Product News', count_labels('Company | Product News'))
print('Treasuries | Corporate Debt', count_labels('Treasuries | Corporate Debt'))
print('Dividend', count_labels('Dividend'))
print('Earnings', count_labels('Earnings'))
print('Energy | Oil', count_labels('Energy | Oil'))
print('Financials', count_labels('Financials'))
print('Currencies', count_labels('Currencies'))
print('General News | Opinion', count_labels('General News | Opinion'))
print('Gold | Metals | Materials', count_labels('Gold | Metals | Materials'))
print('IPO', count_labels('IPO'))
print('Legal | Regulation', count_labels('Legal | Regulation'))
print('M&A | Investments', count_labels('M&A | Investments'))
print('Macro', count_labels('Macro'))
print('Markets', count_labels('Markets'))
print('Politics', count_labels('Politics'))
print('Personnel Change', count_labels('Personnel Change'))
print('Stock Commentary', count_labels('Stock Commentary'))
print('Stock Movement', count_labels('Stock Movement'))

Analyst Update 7023
Fed | Central Banks 3149
Company | Product News 44401
Treasuries | Corporate Debt 2514
Dividend 1208
Earnings 16595
Energy | Oil 6434
Financials 17857
Currencies 3547
General News | Opinion 12468
Gold | Metals | Materials 2342
IPO 833
Legal | Regulation 7356
M&A | Investments 8724
Macro 11210
Markets 24264
Politics 4619
Personnel Change 2690
Stock Commentary 24364
Stock Movement 19915


In [170]:
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 5000
predictions = np.asarray(predictions)

In [172]:
currencies_dataset = financial_news_dataset.loc[predictions=='Currencies', :]
currencies_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
123,NWSA,Financial news,USD Extends Losses Against Major Counterparts,The U S dollar extended losses against the majority of its counterparts as investors continue to speculate whether the Federal Reserve will increase its asset purchasing program by 45 billion per month after Operation Twist expires However some analysts believe the likelihood this will take place is slim as the employment sector issued stellar reports in recent days The greenback weakened further as risk appetite improved following the release of better than expected data out of the eur...,2012-12-12
493,NWSA,Financial news,Euro Moves Lower Amid ECB Speculations,Speculations that the European Central Bank ECB is considering cutting eurozone interest rates in the near future turned the EUR USD moderately bearish during mid day trading yesterday A lack of significant international economic indicators resulted in little other movement among major currencies and commodities over the course of the day Today traders will want to pay attention to the eurozone Retail Sales and Unemployment Rate figures both scheduled to be released at 10 00 GMT If ei...,2013-01-08
1071,NWSA,Financial news,Euro Soared On LTRO News As JPY CAD AUD Dived,Euro finally broke out of recent range last week and ended as the strongest currency on improvement in the banking system as well as confidence Meanwhile Canadian dollar was the weakest one following BoC statement that dented any hope for rate hike and deeper selling was seen after tame inflation reading Aussie and Sterling were both weighed down by disappointing economic data And in spite of strong risk appetite as seen in rally in stock markets CAD AUD and GBP were not supported Som...,2013-01-28
1341,CME,Financial news,GBP USD Maintaining Momentum And Pushing Towards Critical 1 300,Retail currency traders can t seem to make up their minds which way they are going with the USD The GBP USD pair is currently flying high at 1 2902 well ahead of the 50 day moving average and the 200 day moving average This boost took place when Prime Minister May announced that the UK would be holding general elections on 8 June 2017 The effect this had on the sterling was dramatic sending it from around 1 250 to 1 280 in short order \nThat the GBP USD pair has managed to maintain mome...,2017-05-18
2051,NWSA,Financial news,EUR USD Touches Four Week High At 1 31,The last 24 hours has seen the Euro experience some volatility and surge higher to a 4 week high just above 1 31 In doing so it has been able to clear the short term resistance level at 1 3050 The Euro did very well towards the end of last week as it used the solid support at 1 28 and surged higher not only through 1 29 but also the significant level of 1 30 before just easing back to close out the week Over the last couple of weeks it has made several attempts to move through the 1 30 ...,2013-06-04
2364,NWSA,Financial news,Daily FX Analysis Euro Surprisingly Well Bid Despite Cyprus,Major focus is stil on Cyprus and the Euro as they really need to close a deal before the banks re open on Tuesday Even if there is a deal by Tuesday would this limit a run on the banks or has the damage already been done Potentially banks could collapse anyway Even with the support of a new bailout courtesy of the Eurogroup or Russians depositors and wealthy individuals that removed all of their money could have caused a bank run This becomes rather academic For traders the Euro has...,2013-03-21
2573,AA,Financial news,Pound Snapped A Three Day Decline Against The USD,EUR USDThe euro advanced from a one week low against the dollar The 17 nation currency also gained after a Federal Reserve regional business survey said the U S economy expanded modestly last month The euro declined earlier after meetings of European finance ministers this week failed to reassure investors the euro region s sovereign debt crisis will be contained Europe s shared currency rose 0 1 percent

In [173]:
currencies_dataset.shape

(3547, 5)

In [174]:
currencies_dataset.to_csv('currencies_dataset.csv', sep=',', index=False)

In [175]:
analyst_update_dataset = financial_news_dataset.loc[predictions=='Analyst Update', :]
analyst_update_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
107,GOOGL,Financial news,Citigroup Remains Bullish on Facebook,Shares of social media giant Facebook NASDAQ shot up about 0 90 in trading on Friday April 3rd after Citigroup analyst reiterated a Buy rating on the stock and raised his price target from 91 to 97 The analyst reasoned his bullish rating on Facebook s NASDAQ FB revenue potential for the next three years The analyst points out that acquired companies WhatsApp and Instagram are two of the drivers that will help increase Facebook s potential future revenue and user growth He noted ...,2015-04-05
119,CACC,Financial news,Ally Financial s Ratings Upgraded By Moody s Outlook Stable,Ally Financial s NYSE ALLY ratings have been upgraded by Moody s Investors Service The outlook for the company remains stable \nAlly Financial s issuer rating witnessed a progress from Ba3 to Ba2 Further its senior unsecured medium term note program and senior unsecured regular bond debenture rating have been raised from Ba3 to Ba2 \nReasons Behind Rating Upgrade\nAlly Financial witnessed continued improvement in its funding profile backed by the stabilized performance in the auto por...,2019-02-11
328,QCOM,Financial news,Northland Capital upgrades Qualcomm on 3D sensor promise,Northland Capital upgrades Qualcomm NASDAQ QCOM from Market Perform to Outperform with a 62 50 price target Analyst Tom Sepenzis sees the company as well positioned to benefit from the 3 D sensing tech Qualcomm is developing with partner Himax for use in smartphones Sepenzis says Qualcomm could benefit in several ways from SLiM adoption among handset customers including licensing revenue and product revenue and notes that handset providers wanting t...,2017-09-15
544,BAC,Financial news,BofAML ups Nvidia price target to near Street high,Bank of America Merrill Lynch NYSE BAC raises its Nvidia NASDAQ NVDA price target from 185 to 210 and reiterates a Buy rating \nAnalyst Vivek Arya calls Nvidia a top pick and says the firm s positive view on the company is based on its underappreciated transformation from a traditional PC graphics vendor into a supplier into high end gaming enterprise graphics cloud accelerated computing and automotive markets \nThe analyst sees AI as a major potential tailwind for Nvidia \nAry...,2017-09-18
1009,AAPL,Financial news,Apple to gain unconditional EU approval for Shazam buy sources,By Foo Yun Chee BRUSSELS Reuters Apple O AAPL is set to win unconditional EU antitrust approval for its planned acquisition of British music discovery app Shazam two people familiar with the matter said on Wednesday The deal announced in December last year would help the iPhone maker better compete with Spotify N SPOT the industry leader in music streaming services Shazam identifies songs when a smartphone is pointed at an audio source The European Commission opened a full sca...,2018-08-22
1598,BBBY,Financial news,Bed Bath Beyond s Prospects Bleak Sell The Stock Now,Shares of Bed Bath Beyond Inc NASDAQ BBBY a leading operator of specialty retail stores lost nearly 13 over the past six months and estimates are witnessing downward revisions Let s delve deeper and try to find out what is taking this Zacks Rank 4 Sell company down the hill After witnessing a positive earnings surprise of 6 1 in the final quarter of fiscal 2015 Bed Bath Beyond began fiscal 2016 on a dismal note with both its top and bottom lines lagging estimates in the fi...,2016-06-28
1775,BA,Financial news,Trump calls Boeing a very disappointing company as 737 Max crisis grows,Trump called Boeing a big disappointment after the company pushed back the 737 Max s return to service The crisis over Boeing s 737 Max planes after two crashes has already cost thousands of jobs Treasury Secretary Steven Mnuchin recently estimated it could cost half a point of economic growth,2020-01-22
2496,JPM,Financial news,JPMorgan Says Should ve Downgraded Brazil Stocks But Too Late,Bloomberg JPMorgan Chase Co NYSE JPM sai

In [176]:
analyst_update_dataset.shape

(7023, 5)

In [177]:
analyst_update_dataset.to_csv('analyst_update_dataset.csv', sep=',', index=False)

In [178]:
central_banks_dataset = financial_news_dataset.loc[predictions=='Fed | Central Banks', :]
central_banks_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
64,AAPL,Financial news,U S plan to repatriate billions will have limited Hong Kong impact HKMA,HONG KONG Reuters A U S push to encourage its companies to repatriate billions of dollars parked overseas could trigger capital outflows out of Hong Kong though such funds could eventually return to the territory the head of the city s monetary authority said on Monday Low tax Asian financial hubs such as Hong Kong and Singapore have historically benefited from U S multinational giants parking growing cash balances overseas to avoid paying higher tax bills at home But U S Presiden...,2017-02-06
295,SPGI,Financial news,Markets Wait To See If ECB Draghi Adds To Views On Resolving EU Crisis,Economic Data DE Germany August GfK Consumer Confidence Survey 5 9 v 5 8e DE Germany June Import Price Index M M 1 5 v 0 9 e Y Y 1 3 v 1 9 e SE Sweden July Consumer Confidence 5 6 v 2 5e Manufacturing Confidence 2 v 5e Economic Tendency Survey 96 1 v 98 0e SE Sweden June Household Lending Y Y 4 5 v 4 5 e SE Sweden June PPI M M 0 2 v 0 2 e Y Y 0 4 v 0 4 e SE Sweden June Trade Balance SEK 9 3B v 9 3Be SE Sweden June Unemployment Rate 8 8 v 8 9 e EU Euro Zone ...,2012-07-26
337,BLK,Financial news,ECB extends deadline for binding bids for Italy s Carige to mid May sources,MILAN Reuters The European Central Bank has pushed back a deadline to submit binding bids for troubled Italian lender Carige to mid May so as to give a specialist fund run by BlackRock NYSE BLK more time two sources familiar with the matter said The BlackRock fund has remained as the only buyer still known to be considering a bid One of the sources said it needed more time to study a possible offer given all the moving parts involved Carige was placed under special administration...,2019-04-11
491,BAC,Financial news,After sprinting to aid rupiah Indonesia c bank could struggle in a marathon,By Maikel Jefriando and Fransiska Nangoy JAKARTA Reuters Bank Indonesia has done more than any Asian peer to defend its currency amid a global rout in emerging markets but the fact that the rupiah has kept slipping suggests the central bank would struggle to maintain the pace in a prolonged battle Like many developing markets with current account or trade deficits Southeast Asia s largest economy is hostage to forces outside its control including rising U S interest rates higher oi...,2018-07-27
630,MS,Financial news,China to Unleash 108 Billion in Reserves Cut for Most Banks,Bloomberg China s central bank will cut the amount of cash some lenders must hold as reserves unlocking about 700 billion yuan 108 billion of liquidity as it seeks to control leverage and support smaller companies \nThe required reserve ratio for some banks will drop by 0 5 percentage point effective July 5 the People s Bank of China said on its website That s the day before the U S and China are scheduled to impose tariffs on each other and the cut should help companies affec...,2018-06-24
989,WMB,Financial news,Fed s Williams Central bank will adjust response to money market volatility as appropriate,By Jonnelle Marte NEW YORK Reuters New York Federal Reserve President John Williams NYSE WMB said Thursday the central bank is learning as it works to increase liquidity in the banking system and that it will adjust its approach as needed One early lesson from the Fed s daily interventions in money markets is that a standing repo facility which would allow banks to convert Treasury holdings to cash might not be enough on its own to make sure banks have the liquidity they need Will...,2019-10-17
1414,VALE,Financial news,What Impact Will The Fed s Glass Half Full Approach Have,It s been an interesting session and the traders and investors are still trying to price markets accordingly The event risk started with US CPI ex food coming in at a below par 1 7 and this really started the ball rolling with a keen focus on inflationary trends in the FOMC statement and through Janet Yellen s 

In [179]:
central_banks_dataset.shape

(3149, 5)

In [180]:
central_banks_dataset.to_csv('central_banks_dataset.csv', sep=',', index=False)

In [181]:
product_news_dataset = financial_news_dataset.loc[predictions=='Company | Product News', :]
product_news_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
3,GM,Financial news,GM to respond by January 7 to Canada union proposals to save Ontario plant,Reuters General Motors Co NYSE GM will respond by Jan 7 to Canadian union proposals to keep an Ontario auto assembly plant open the head of Canada s largest private sector union said on Thursday While GM has gone too far in planning to close the plant the U S automaker didn t slam the door to union efforts to save it Unifor head Jerry Dias told reporters after meeting with GM executives in Detroit \nGM said in November it will close five North American plants next year inc...,2018-12-20
6,WFC,Financial news,Wells Fargo plans 1 000 U S job cuts,By Imani Moise Reuters Wells Fargo NYSE WFC Co said on Thursday it notified about 1 000 employees in its Consumer Lending and Payments Virtual Solutions and Innovations groups of plans to eliminate their positions The layoffs are consistent with the bank s previously announced plans to reduce headcount by up to 10 percent by 2020 spokesman Tom Goyda said Wells Fargo the No 4 U S bank by assets had about 262 000 employees as of Sept 30 Most of those affected received 60 da...,2018-11-15
10,ROK,Financial news,ManpowerGroup Collaborates With Rockwell To Train Veterans,Leading staffing company ManpowerGroup Inc NYSE MAN recently collaborated with Rockwell Automation Inc NYSE ROK to train 1 000 military veterans for advanced manufacturing jobs Both the companies will work closely together utilizing existing programs to train veterans in advanced digital manufacturing jobs and other higher tech roles in the industry Per a survey conducted by ManpowerGroup most U S employers anticipate that at a higher skill level automation will increase the h...,2017-06-13
48,SRPT,Financial news,Sarepta Looks To Focus On Exondys 51 DMD Pipeline In 2017,We issued an updated research report on Cambridge MA based Sarepta Therapeutics Inc NASDAQ SRPT on Jun 28 Sarepta is a commercial stage biopharmaceutical company that focuses on the discovery and development of RNA based therapeutics targeting rare and infectious diseases especially Duchenne muscular dystrophy DMD So far this year Sarepta s share price has outperformed the Zacks categorized industry The stock has surged 21 6 compared with an increase of 10 0 for the broader in...,2017-06-29
87,AAPL,Financial news,HSBC retail head sees more bank partnerships with financial tech firms,TORONTO Reuters HSBC s L HSBA retail banking and wealth management head John Flint expects banks to partner more with financial technology fintech companies in the coming years to help meet changing demands from customers Banks are increasingly teaming up with fintech companies previously seen a threat as they look to respond to a sharp increase in customers banking online and to take advantage of opportunities created by new technologies such as blockchain I think there s an aw...,2016-09-12
93,WYNN,Financial news,Sunny day in Hong Kong sets up Melco Resorts,Melco Resorts Entertainment NASDAQ MLCO is higher in early trading after the Macau sector saw an upbeat day in Hong Kong off of new revenue forecasts Nomura came in with an estimate of 15 to 17 Macau revenue growth in May to follow closely on a strong outlook from Aegis Capital The firm thinks some of the concerns on regulations cutting off growth are overstated Galaxy Entertainment OTCPK GXYEF was up 4 01 in Hong Kong today while Wynn Macau OTCPK WYNMF OTCPK WYNMY WYNN ...,2017-05-23
99,AGN,Financial news,Allergan Drops Stops Textured Breast Implant Sales In Europe,Allergan plc NYSE AGN announced that it has suspended the sale of its textured breast implants and tissue expanders and will further discontinue the supply of any remaining products in Europe reportedly due to a cancer risk The decision was taken following a compulsory recall request from French regulatory authority Agence Nationale de S curit du M dicament ANSM after the product s CE Mark certification expired Shares of Al

In [182]:
product_news_dataset.shape

(44401, 5)

In [183]:
product_news_dataset.to_csv('product_news_dataset.csv', sep=',', index=False)

In [184]:
treasuries_dataset = financial_news_dataset.loc[predictions=='Treasuries | Corporate Debt', :]
treasuries_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
682,AAPL,Financial news,Citigroup To Redeem Subordinated Notes Worth C 481 5 Million,Citigroup Inc NYSE C recently announced 100 redemption of its outstanding 4 650 Fixed Rate Floating Rate Subordinated Notes due October 2022 The principal amount of the notes was C 481 5 million \nThe notes are redeemable on Oct 11 2017 Citigroup stated that the total amount to be paid on redemption includes the entire principal amount of the notes plus accrued and unpaid interest thereof amounting to approximately C 11 million Citibank N A is the paying agent for the subordi...,2017-09-10
817,TSLA,Financial news,Tesla raises borrowing capacity for car leases to 1 1 billion,Reuters Tesla Inc O TSLA said on Friday it has increased its borrowing capacity for a car lease program to 1 1 billion from 600 million The move comes as the electric car maker spends heavily to fix production bottlenecks of its new Model 3 sedan The company increased the borrowing capacity under certain warehouse agreements the company said in a filing A spokesman said it was related to Tesla s car lease program \nIn August Tesla said it would raise about 1 5 billion through ...,2017-10-20
976,MS,Financial news,Week That Spooked Bond Markets Sent Traders Scrambling for Exits,Bloomberg Volatility in the U S Treasury market appears to have forced traders out of what just a week ago appeared to be surefire wagers on higher yields and a flatter curve \nFlight to quality as U S stocks slumped pulled the 10 year note s yield from a four year high of 3 259 percent on Tuesday to as low as 3 124 percent on Thursday Apparently that was enough for one trader to abandon a 2 5 million wager targeting a move to 3 60 percent by December Placed last week the bet took...,2018-10-12
1471,BAC,Financial news,US 10 Year Yields Fall From 12 Month High,Treasuries advanced after 10 year U S government securities suffered their biggest one day decline since October 2011 yesterday as the highest yields in 13 months helped attract buyers Securities in the Bank of America Merrill Lynch Global Broad Market Index have fallen 1 3 percent in May poised for the steepest loss since April 2004 while Treasuries have dropped 1 9 percent this month the indexes show Yields rose yesterday after a report showed U S consumer confidence climbed to a five...,2013-05-30
1902,MMM,Financial news,Scandi FX Volatility Monitor Sell 3M EUR SEK Straddle With Strike Skewed,Implied volatility is little changed from last week in the Scandi FX sphere Realised volatility has declined substantially in both NOK and SEK crosses which should curb upside potential in implied volatilities in the near term in the absence of accelerating risk off jitters \nFrom a risk reward point of view we still see value in selling 3M NYSE MMM EUR SEK straddle with the strike skewed on the upside relative to the FX forward However we note that political uncertainty in Sweden rem...,2019-01-14
1951,BAC,Financial news,Evergrande s debt burden just keeps on growing squeezing shareholders,By Clare Jim and Umesh Desai\nHONG KONG Reuters Building up the second biggest corporate debt pile in China does come at a cost \nChina Evergrande Group HK 333 the nation s No 2 real estate developer reported last week that its borrowings grew to 57 billion by the end of June including so called perpetual bonds Only state owned Petrochina HK 0857 owes more \nThe crushing impact of that burden became clear in its first half results as despite reporting a 12 6 percent jump in sal...,2016-09-04
2454,JPM,Financial news,JPMorgan pledges up to 2 million to fight racism support human rights,By David Henry NEW YORK Reuters JPMorgan Chase Co N JPM said on Monday it will contribute up to 2 million to fight racism and support human rights in light of a white nationalist rally in Charlottesville Virginia that led to violence and the death of a protester The sum includes 500 000 each to the Southern Poverty Law Center and the Anti De

In [185]:
treasuries_dataset.shape

(2514, 5)

In [186]:
treasuries_dataset.to_csv('treasuries_dataset.csv', sep=',', index=False)

In [187]:
dividend_dataset = financial_news_dataset.loc[predictions=='Dividend', :]
dividend_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
256,BAC,Financial news,Bank of America Corporation Deposit Shs Perp Pfd Shs Series E declares 0 2556 dividend,Bank of America Corporation Deposit Shs Perp Pfd Shs Series E NYSE BAC PE declares 0 2556 share quarterly dividend in line with previous \nForward yield 4 13 \nPayable Feb 18 for shareholders of record Jan 21 ex div Jan 20 \nSee BAC PE Dividend Scorecard Yield Chart Dividend Growth,2020-01-17
412,AVB,Financial news,AvalonBay Communities declares 1 47 dividend,AvalonBay Communities NYSE AVB declares 1 47 share quarterly dividend 3 5 increase from prior dividend of 1 42 Forward yield 3 45 Payable April 16 for shareholders of record March 29 ex div March 28 Now read,2018-01-31
1202,SNV,Financial news,Synovus declares 0 15 dividend,Synovus NYSE SNV declares 0 15 share quarterly dividend in line with previous Forward yield 1 46 Payable Oct 2 for shareholders of record Sept 21 ex div Sept 20 Now read,2017-09-08
1210,SPGI,Financial news,S P Global declares 0 41 dividend,S P Global NYSE SPGI declares 0 41 share quarterly dividend in line with previous Forward yield 1 12 Payable Sept 12 for shareholders of record Aug 28 ex div Aug 24 Now read,2017-06-27
2615,SO,Financial news,Southern declares 0 60 dividend,Southern NYSE SO declares 0 60 share quarterly dividend in line with previous Forward yield 5 02 Payable Sept 6 for shareholders of record Aug 20 ex div Aug 17 Now read,2018-07-16
3084,ENTG,Financial news,Entegris declares 0 08 dividend,Entegris NASDAQ ENTG declares 0 08 share quarterly dividend in line with previous \nForward yield 0 6 \nPayable Feb 19 for shareholders of record Jan 29 ex div Jan 28 \nSee ENTG Dividend Scorecard Yield Chart Dividend Growth,2020-01-15
3180,ROP,Financial news,Roper s Board Approves 11 Hike In Quarterly Dividend Rate,Roper Technologies Inc NYSE ROP on Nov 15 announced that it is rewarding shareholders in the form of a hike in the quarterly dividend rate This marks its 27th consecutive year of dividend rate increase We believe that such shareholder friendly policies of the company reflect a strong cash position Inside the HeadlinesAs revealed Roper s board of directors approved an 11 or 5 cents per share hike in the quarterly dividend rate which now moved from 46 25 cents to 51 25 cents On an...,2019-11-18
3459,PHM,Financial news,PulteGroup hikes dividend by 13 approves 300 million stock buyback,Investing com Coming off a strong third quarter PulteGroup Inc N PHM said on Wednesday evening that its Board of Directors has raised its quarterly dividend and authorized a 300 million stock repurchase plan effective early next year \nPulteGroup one of the largest homebuilding companies in the U S declared a quarterly cash dividend of 0 09 per common share payable on January 5 to shareholders of record at the close of business on December 15 The 0 09 per share dividend represe...,2015-12-02
4265,AAPL,Financial news,Dow 30 Stock Roundup UnitedHealth Hikes Quarterly Dividend By 20 Microsoft Confirms GitHub Buyout,The Dow enjoyed a week of steady gains with trade concerns largely taking a backseat Worries related to trade were only evident on Tuesday after Mexico imposed tariffs on a wide range of U S products A spike in yields boosted financials on Wednesday Tech stocks continued to make steady gains until investors indulged in profit taking on Thursday Last Week s PerformanceThe Dow gained 0 9 last Friday on a stronger than expected jobs report shrugging off trade off fears to some extent The...,2018-06-07
6331,INTC,Financial news,Microsoft boosts dividend 11 percent adds new directors,SEATTLE Reuters Microsoft Corp O MSFT on Tuesday announced the smallest percentage increase in its quarterly dividend since 2009 and appointed new directors to replace two board members who decided not to seek re election \nThe increase 11 percent to 31 cents per share is in line with the software company s revenue growth last fiscal year but half of last year s 22 percent

In [188]:
dividend_dataset.shape

(1208, 5)

In [189]:
dividend_dataset.to_csv('dividend_dataset.csv', sep=',', index=False)

In [190]:
earnings_dataset = financial_news_dataset.loc[predictions=='Earnings', :]
earnings_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
49,PCAR,Financial news,Should You Sell PACCAR PCAR Before Earnings,Investors are always looking for stocks that are poised to beat at earnings season and PACCAR Inc NASDAQ PCAR may be one such company The firm has earnings coming up pretty soon and events are shaping up quite nicely for their report That is because PACCAR is seeing favorable earnings estimate revision activity as of late which is generally a precursor to an earnings beat After all analysts raising estimates right before earnings with the most up to date information possible is ...,2017-10-23
56,AVB,Financial news,REITs To Watch For Earnings Early Next Week AVB AGNC More,The Q3 earnings season for real estate investment trust REIT stocks has already commenced and Monday will mark the beginning of one of the busiest weeks of the season Student REIT American Campus Communities Inc NYSE ACC residential REIT Avalonbay Communities Inc NYSE AVB retail REITs Agree Realty Corp NYSE ADC and Brixmor Property Group Inc NYSE BRX as well as mortgage REIT AGNC Investment Corp NASDAQ AGNC are slated to report their earnings on Oct 24 20...,2016-10-20
85,AAPL,Financial news,Game On What to Expect from Nike s Earnings,All eyes will be on NYSE NKE after today s close The Estimize community expects the sportswear giant to post EPS of 0 88 That s four cents higher than what the Street is anticipating This would show year over year earnings growth of 16 the third consecutive quarter of double digit growth yet sliding a bit from the 25 seen in the last two quarters The Estimize consensus for revenues of 7 71B also tops the Wall Street consensus of 7 64B Revenue growth of 10 would mark the fift...,2015-03-19
198,TTM,Financial news,Group 1 Automotive GPI Lags Q3 Earnings And Revenues,Share price of Group 1 Automotive Inc NYSE GPI increased 0 9 to 59 66 on Oct 21 after the company reported third quarter 2016 results Group 1 Automotive recorded a 2 6 year over year increase in earnings per share to 1 96 in the third quarter of 2016 that missed the Zacks Consensus Estimate of 2 08 Adjusted net income declined 8 7 to 42 million from 46 million in the year ago quarter On reported basis Group 1 Automotive recorded a net income of 35 4 million or 1 65 per sha...,2016-10-25
243,AAPL,Financial news,Will Revenue Growth Favor Papa John s PZZA Q4 Earnings,Papa John s International Inc NASDAQ PZZA is scheduled to report fourth quarter 2017 results on Feb 27 after market close The company s continued international expansion plans strategic partnerships and strong digital platform are likely to reflect in fourth quarter results encouraging top line performance However higher costs associated with labor startups and transition might dent earnings in the to be reported quarter Notably shares of Papa John s have declined 33 3 in the pa...,2018-02-19
274,BKR,Financial news,Can EOG Resources EOG Spring A Surprise In Q1 Earnings,Upstream energy player EOG Resources Inc NYSE EOG is expected to release first quarter 2017 financial results on May 9 Last quarter the company posted a positive earnings surprise of 93 75 In the last four quarters the company s average earnings surprise was a positive 22 94 Let s see how things are shaping up for this announcement Factors Influencing This Past QuarterCompared with the year earlier quarter the pricing scenario of oil was much better during the first quarter ...,2017-05-03
319,AAPL,Financial news,FANG s Must See Earnings Charts,This is the week that investors love \nEven though it s not the busiest week in earnings season with just about 400 companies reporting it includes some of the biggest most glamorous technology and social media companies on the planet \nThese are the stocks we all own \nThe so called FANG stocks and the FANG like stocks like Alibaba have seen their shares soar in 2017 but many are worried about valuation and the possibility of a melt up in some of these stocks \nJust what do their ...,2018-01

In [191]:
earnings_dataset.shape

(16595, 5)

In [192]:
earnings_dataset.to_csv('earnings_dataset.csv', sep=',', index=False)

In [194]:
energy_oil_dataset = financial_news_dataset.loc[predictions=='Energy | Oil', :]
energy_oil_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
33,BAC,Financial news,Guess What Happened The Last Time The Price Of Oil Crashed Like This,There has only been one other time in history when the price of oil has crashed by more than 40 dollars in less than 6 months The last time this happened and the beginning of that oil price crash preceded the great financial collapse that happened later that year by several months Well now it is happening again but this time the stakes are even higher When the price of oil falls dramatically that is a sign that economic activity is slowing down It can also have a tremendously de...,2014-12-01
115,GS,Financial news,Oil Peeks Above 57 a Barrel Amid Conflicting Signals on Glut,Bloomberg Crude rose after one of the most influential voices in world oil markets issued mixed signals about the direction of supply and demand \nFutures closed 0 8 percent higher in New York The International Energy Agency increased its forecast for 2018 production by nations outside the OPEC cartel even as it expressed confidence the Saudi Arabia led group will significantly curb a global oversupply Traders were also confronted with swelling U S gasoline stockpiles amid a four wee...,2017-12-14
163,SLB,Financial news,Energy sector on track for worst one day drop since March,U S crude oil plunges 3 to 42 85 bbl hitting its lowest level since mid November and retreating to bear market territory defined as a drop of at least 20 from a recent peak The fresh leg lower comes on signs of rising output from Nigeria and Libya two OPEC members currently exempt from cutting supply John Kilduff founding partner at energy hedge fund Again Capital tells CNBC that oil prices most definitely are heading to 40 and likely into the upper 30s The gloom means the Energ...,2017-06-20
181,STLD,Financial news,Steel Stocks Weigh On Markets,This morning many of the leading steel stocks are declining lower at the start of the trading day United States Steel Corp X is considered to be the leading stock in the sector Today X stock is trading lower by 60 cents to 25 11 a share The daily chart for X stock has a lot of near term resistance around the 26 50 level Short term traders can watch for intra day support around the 25 00 and 24 58 levels Some other leading steel stocks that are also coming under selling pressure ...,2013-01-08
494,C,Financial news,Commodity Sector Outlook Got You Down Consider This Metal,By Commodities have started to seriously bore Wall Street That s because commodity prices are expected to hold steady at best this year and many are expected to fall Production is generally seen as keeping pace with or exceeding demand and inventories of a number of commodities are not low enough to bolster prices \nBarclays PLC put it succinctly recently in noting that commodity returns will remain sluggish for some time to come Consider oil the most heavily traded commodit...,2014-01-29
949,GS,Financial news,Brent steady U S crude down dollar gasoline limit oil s downside,By Barani Krishnan NEW YORK Reuters Brent oil erased most of its early losses to trade steady on Thursday on support from a weak dollar and higher gasoline prices while U S crude slipped on pressure from large inventory builds Brent futures LCOc1 were flat at 44 14 a barrel by 11 52 a m EST 1652 GMT after hitting a session low of 43 70 U S crude s benchmark West Texas Intermediate WTI futures CLc1 fell 40 cents to 40 35 after hitting an intraday low of 41 05 The dol...,2015-11-19
1002,XOM,Financial news,UPDATE 9 Oil edges up in choppy trade posts weekly gain,Fitch downgrades of Spain Italy rattle markets\n U S nonfarm payrolls data supportive to oil\n Coming up API oil data 3 30 p m EDT Wednesday\n Recasts updates prices to settlement \n By Robert Gibbons\n NEW YORK Oct 7 Reuters Oil prices edged higher in\nvolatile trading on Friday and posted a weekly gain as\nsupportive U S jobs data and a ratings downgrades of Spain and\nItaly buffeted markets \n Both Brent and U S 

In [195]:
energy_oil_dataset.shape

(6434, 5)

In [196]:
energy_oil_dataset.to_csv('energy_oil_dataset.csv', sep=',', index=False)

In [197]:
financials_dataset = financial_news_dataset.loc[predictions=='Financials', :]
financials_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
39,WMB,Financial news,Williams Companies Inc The WMB Q4 Earnings And Revenues Lag Estimates,Williams Companies Inc The NYSE WMB came out with quarterly earnings of 0 19 per share missing the Zacks Consensus Estimate of 0 22 per share This compares to earnings of 0 20 per share a year ago These figures are adjusted for non recurring items \nThis quarterly report represents an earnings surprise of 13 64 A quarter ago it was expected that this pipeline operator would post earnings of 0 18 per share when it actually produced earnings of 0 24 delivering a surprise of ...,2019-02-13
55,APA,Financial news,Apache APA Q2 Earnings And Revenues Surpass Estimates,U S energy firm Apache Corporation NYSE APA reported second quarter 2019 earnings per share excluding one time items of 11 cents ahead of the Zacks Consensus Estimate of 4 cents The outperformance stems from robust international production However Apache s bottom line witnessed a fall from the year ago quarter s adjusted profit of 50 cents on lower commodity price realizations Revenues of 1 6 billion were 1 4 above the Zacks Consensus Estimate but was 16 9 lower than the secon...,2019-08-01
160,NWSA,Financial news,Volume Explodes On Accusations Of Accounting Sham,Ebix Inc NASDAQ EBIX is an international supplier of software and e commerce solutions to the insurance industry The news accusations of accounting fraud In fact the term sham has been thrown around Gotham City Research LLC has claimed in a note that the company s accounting is unreliable inaccurate and incomplete its tax strategy is a sham and its stock deserves to be halted per A Barron s article The stock shows up on the elevated vol scan and high option volume relativ...,2013-02-23
290,NOC,Financial news,Northrop ups guidance again,Diluted EPS increased 11 to 3 15 from 2 85 in the second quarter of 2016 Sales by segment Aerospace 14 Mission 3 Technology 3 Raised outlook for 2017 Diluted EPS of 12 10 12 40 vs 11 80 12 10 on sales of approximately 25B NOC 0 2 premarketQ2 resultsNow read,2017-07-26
303,TDC,Financial news,Teradata TDC Q1 Earnings Beat Estimates,Teradata TDC came out with quarterly earnings of 0 22 per share beating the Zacks Consensus Estimate of 0 19 per share This compares to earnings of 0 19 per share a year ago These figures are adjusted for non recurring items \nThis quarterly report represents an earnings surprise of 15 79 A quarter ago it was expected that this data management company would post earnings of 0 44 per share when it actually produced earnings of 0 49 delivering a surprise of 11 36 \nOver the last...,2019-05-02
333,HD,Financial news,Why Home Depot HD Is Poised To Beat Earnings Estimates Again,Looking for a stock that has been consistently beating earnings estimates and might be well positioned to keep the streak alive in its next quarterly report Home Depot NYSE HD which belongs to the Zacks Building Products Retail industry could be a great candidate to consider \nWhen looking at the last two reports this home improvement retailer has recorded a strong streak of surpassing earnings estimates The company has topped estimates by 4 63 on average in the last two quarters...,2019-08-18
364,CHTR,Financial news,WWE Q3 Earnings To Gain From Subscriber Growth TV Right Fees,World Wrestling Entertainment Inc NYSE WWE is scheduled to report third quarter 2017 financial numbers before the opening bell on Oct 26 In the previous quarter the company beat the Zacks Consensus Estimate by 16 7 Notably the company s earnings have missed the Zacks Consensus Estimate in two of the trailing four quarters by an average of 8 2 Here s a discussion of the determinants of the second quarter results Factors Influencing This QuarterAfter generating record revenues in ...,2017-10-16
389,ALGT,Financial news,Gol Linhas Q4 Unit Revenue View Upbeat On Capacity Discipline,Gol Linhas Aereas Inteligentes S A NYSE GOL has provided a bullish view on its upcoming fourth quar

In [198]:
financials_dataset.shape

(17857, 5)

In [199]:
financials_dataset.to_csv('financials_dataset.csv', sep=',', index=False)

In [200]:
general_news_dataset = financial_news_dataset.loc[predictions=='General News | Opinion', :]
general_news_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
52,JPM,Financial news,Will The Zombie Apocalypse End The Real Estate Recovery,We covered the investment wisdom of phantoms yesterday Today we re moving on to vampires and zombies And no this isn t some spoof in honor of Halloween It s totally legit Well respected financial research firms swear that a legion of zombies and vampires a couple hundred thousand strong threaten to suck the life out of the housing market according to RealtyTrac s Vice President Daren Blomquist One of the more prescient market forecasters in recent years is even predicting a do...,2013-10-31
89,WMB,Financial news,Jets down Darnold draw testy Browns in primetime,The New York Jets will look to avoid an 0 2 start when they go to battle without Sam Darnold The quarterback is sidelined indefinitely due to mononucleosis and the Jets will start Trevor Siemian when they host the Cleveland Browns on Monday night Darnold was losing weight and lacking energy so the team ran some tests that detected the illness Jets coach Adam Gase offensive coordinator Dowell Loggains and a team trainer informed Darnold late Wednesday night that he couldn t play against...,2019-09-12
128,BHC,Financial news,Ackman s fund falls 7 percent in March hurt by Valeant,BOSTON Reuters Billionaire investor William Ackman s Pershing Square NYSE SQ Holdings Ltd fund lost 7 1 percent last month hurt largely by Valeant Pharmaceuticals TO VRX roughly 60 percent tumble in March March s poor performance leaves Ackman s fund with a 25 6 percent loss for the year a person familiar with its returns said on Friday This marks a dramatic turnaround from Pershing Square s 40 percent gain in 2014 \nPershing Square owns a 9 percent stake in Valeant and the co...,2016-04-01
234,GS,Financial news,John Whitehead former leader of Goldman Sachs dies at 92,By Doina Chiacu WASHINGTON Reuters John Whitehead a former senior partner and co chairman of Goldman Sachs who helped make it a top tier Wall Street firm and led its international expansion has died the investment bank said on Saturday He was 92 Whitehead joined Goldman Sachs in 1947 and worked his way to the highest rung of its corporate ladder before leaving after 38 years to become a deputy secretary of state under U S President Ronald Reagan He was a chairman of the Federal Re...,2015-02-07
420,ANF,Financial news,Bubbles Anyone,Investment bubbles are not hard to spot There are times when insanity trumps everything and prices move higher and higher in spite of reality s tethers Bubbles float on insanity Wild speculation Irrational expectations Hope \nThis week that ended 11 20 15 is a good example of another such bubble \nMass murderers struck Paris and Mali More retailers reported slower to no growth The Fed issued more warnings of a December rate increase Japan lapsed into another recession Europe is so s...,2015-11-22
738,SLB,Financial news,Day Ahead Top 3 Things to Watch,Investing com Here s a preview of the top 3 things that could rock markets tomorrow \n1 Netflix Slumps After Hours\nNetflix NASDAQ NFLX could weigh on tech stocks tomorrow after reporting mixed earnings after the bell today \nThe consensus on Wall Street seemed to be that the numbers weren t so hot as the stock fell more than 3 after hours \nBut guidance was also mixed so look for lots of commentary on whether to look on Netflix as half full or half empty \nA lot will depend on whethe...,2019-01-17
897,BA,Financial news,3 Defense Picks If Russia Intervenes In U S Syria Crisis,The United States along with its French and UK allies launched a series of airstrikes on Syria over the last weekend 105 missiles were fired on three facilities involved in research or storage of chemical weapons in western Syria On Apr 7 Duma was a witness to a chemical attack which claimed several lives The recent attack is believed to be a response to the same The world once again witnessed fresh tension following which U S defense stocks gained substantially as

In [201]:
general_news_dataset.shape

(12468, 5)

In [202]:
general_news_dataset.to_csv('general_news_dataset.csv', sep=',', index=False)

In [203]:
gold_metals_dataset = financial_news_dataset.loc[predictions=='Gold | Metals | Materials', :]
gold_metals_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
153,WMT,Financial news,Dollar gains gold loses luster amid robust U S data,By Lewis Krauskopf NEW YORK Reuters The U S dollar strengthened on Thursday pressuring gold prices as strong labor market data gave support to a possible hike in U S interest rates later this year U S Treasury yields rose to three week highs ahead of the closely watched U S jobs report due out on Friday In an encouraging sign for the labor market data on Thursday showed the number of Americans filing for unemployment benefits unexpectedly fell last week to near a 43 year low O...,2016-10-06
601,CME,Financial news,Gold prices gained in Asia as Fed views on rates assessed,Investing com Gold gained in Asia on Tuesday with sentiment on expectations for the path of U S interest rates this year they key focus \nOn the Comex division of the New York Mercantile Exchange gold for April delivery rose 0 57 to 1 217 10 a troy ounce \nSilver futures for March delivery gained 0 50 to 15 260 a troy ounce while copper futures for March delivery eased 0 61 to 2 102 a pound \nOvernight gold plummeted more than 1 on Monday amid a broadly stronger dollar as inves...,2016-02-22
1180,NFLX,Financial news,Implied Volatility Surging For Energous WATT Stock Options,Investors in Energous Corporation NASDAQ WATT need to pay close attention to the stock based on moves in the options market lately That is because the Feb 15 2019 2 50 Put had some of the highest implied volatility of all equity options today What is Implied Volatility Implied volatility shows how much movement the market is expecting in the future Options with high levels of implied volatility suggest that investors in the underlying stocks are expecting a big move in one direction ...,2018-12-23
1199,AA,Financial news,Gold remains near 2 year high even as global political outlook improves,Investing com Gold closed virtually flat on Monday in choppy trade as Shinzo Abe s landslide victory in Japan and the likelihood of Theresa May s appointment as David Cameron s replacement in the U K helped reduce global political uncertainty offsetting the rising probability of further easing measures by top central banks worldwide \nOn the Comex division of the New York Mercantile Exchange Gold for August delivery traded between 1 352 00 and 1 376 40 an ounce before settling at ...,2016-07-11
1760,GM,Financial news,Platinum Price May Outperform Gold Silver Based OnSouth African Risk,We are seeing some healthy profit taking in precious metals after making an explosive breakout over the summer Investment demand after QE3 is increasing as investors seek alternatives to fiat currencies which are being devalued by Central Banks all over the world We may see consolidation and volatility in the markets until after the U S Presidential Election when most investors realize that not much will change All over the world governments are looking to boost unhealthy economies and t...,2012-10-21
2739,NWSA,Financial news,Gold prices slide as investors take profits from U S jobs report rally,Investing com Gold prices fell on Monday after investors sold the commodity for profits after Friday s rally the product of weaker than expected jobs numbers that sparked fresh talk that the Federal Reserve will keep monetary policy loose a recipe for rising gold prices On the Comex division of the New York Mercantile Exchange gold futures for June delivery were down 0 23 at USD1 572 35 a troy ounce in U S trading on Monday up from a session low of USD1 567 15 and down from a high of...,2013-04-08
2791,NFLX,Financial news,Implied Volatility Surging For Realogy Holdings RLGY Stock Options,Investors in Realogy Holdings Corp NYSE RLGY need to pay close attention to the stock based on moves in the options market lately That is because the Mar 15 2019 17 50 Put had some of the highest implied volatility of all equity options today What is Implied Volatility Implied volatility shows how much movement the market is ex

In [204]:
gold_metals_dataset.shape

(2342, 5)

In [205]:
gold_metals_dataset.to_csv('gold_metals_dataset.csv', sep=',', index=False)

In [206]:
ipo_dataset = financial_news_dataset.loc[predictions=='IPO', :]
ipo_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
472,WBA,Financial news,Marijuana Banking IPOs Weed Votes Will Hemp Grow Higher Revised,The much awaited Cannabis Banking Bill was finally passed by the House Financial Services Committee on Mar 28 This opens up avenues for legal and effective financing for marijuana companies in the United States Meanwhile Walgreens NASDAQ WBA announced plans to sell weed based products through its drug retail stores Interestingly a number of marijuana companies are expected to go public this year With both Canada and the United States becoming increasingly weed liberal a bump in wee...,2019-04-03
950,BIDU,Financial news,China s Aurora Mobile files for IPO of American sahres,Chinese app services provider Aurora Mobile Limited has filed for an initial public offering of American Depositary Shares The filing comes via Goldman Sachs NYSE GS Asia Credit Suisse SIX CSGN and Deutsche Bank DE DBKGn Securities It s registering a 200M offering and looking to list on Nasdaq under the symbol JG For the three months ended March 31 the company notes revenues of 20 15M and gross profit of 5 5M leading to a net loss of 3 53M The company says it currently...,2018-06-29
1386,BX,Financial news,Auto1 may consider IPO in future but no need for cash now CEO,BERLIN Reuters German used car dealing platform Auto1 said it could seek a public offering in future but a 2018 cash infusion from Japan s Softbank means it has no immediate need for extra funding of its European growth plans Last year s Softbank s deal valued Berlin based Auto1 at 2 9 billion euros 3 27 billion making it one of Germany s top so called tech unicorns It is virtually unknown to consumers except through its used car buying arm Wir Kaufen dein Auto We Buy Your Car i...,2019-03-27
4319,INTC,Financial news,Corporate investors pile into electric vehicle startups,By Paul Lienert DETROIT Reuters Corporate investors from outside the auto industry are placing increasing bets on electric vehicles vying with automakers and suppliers to bankroll startup companies working on everything from advanced batteries to charging devices and all new EVs Some 250 startups involved in some aspect of electrification have attracted more than 20 billion in venture capital notably from a broad array of corporations across multiple industries according to a Reuter...,2019-01-11
6597,EXPE,Financial news,Trivago IPO Are Tech and Travel Rising Again,In a bit of good news for the tech IPO market hotel search engine company Trivago has filed for an IPO which could see the company raise up to 400 million Trivago has not determined the number of shares nor their price and will list their shares on the NASDAQ with the symbol TRVG among other sources reports that Trivago could debut before the end of 2016 and the company appears to want to debut as soon as possible given earlier reports of a Thanksgiving IPO Trivago s debut as well ...,2016-11-18
6722,UBER,Financial news,Crown jewel Aramco stirs loyal Saudi demand for giant IPO,By Marwa Rashad and Stephen Kalin RIYADH Reuters From taxi drivers to clerics Saudis clamoring to own part of state oil giant Aramco went online and to local banks on Sunday at the start of a long delayed share sale for what could be the world s biggest initial public offering Ahmed Sanad 37 who signed up at a Saudi British Bank SABB branch in central Riyadh called the IPO which has dominated conversations on social media in cafes and at family gatherings the talk of the town ...,2019-11-17
6818,AAPL,Financial news,Braeburn Pharmaceuticals files for IPO,Reuters Braeburn Pharmaceuticals Inc filed with U S regulators on Friday to raise up to 150 million in an initial public offering seven months after its implant to treat opioid addiction was approved in the United States \nThe U S Food and Drug Administration in May approved an implant to fight addiction to opioids a class of drugs that includes prescription painkillers and heroin \nBraeburn said it is planning 

In [207]:
ipo_dataset.shape

(833, 5)

In [208]:
ipo_dataset.to_csv('ipo_dataset.csv', sep=',', index=False)

In [209]:
legal_dataset = financial_news_dataset.loc[predictions=='Legal | Regulation', :]
legal_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
26,XRX,Financial news,Icahn Deason urge Xerox shareholders to oppose Fujifilm deal,Reuters Xerox Corp s N XRX plan to sell itself to Japan s Fujifilm Holdings T 4901 has come under further pressure with Carl Icahn and Darwin Deason urging fellow shareholders to oppose the 6 1 billion deal The activist shareholders who own a combined 15 percent of the U S printer and copier maker said the agreement dramatically undervalued Xerox and criticized the deal structure which calls for the U S firm to be combined into the Fuji Xerox joint venture as tortured convo...,2018-02-13
260,MSFT,Financial news,Ex Autonomy CFO sentenced in U S to 5 years prison over Hewlett Packard fraud,By Jonathan Stempel\n Reuters The former chief financial officer of British software company Autonomy was sentenced on Monday to five years in prison after a U S jury found him guilty of fraud over the 11 1 billion sale of Autonomy in 2011 to Hewlett Packard \nSushovan Hussain 55 was also fined 4 million and ordered to forfeit 6 1 million by U S District Judge Charles Breyer in San Francisco The defendant plans to appeal \nHussain and former Autonomy Chief Executive Mike Lynch ar...,2019-05-14
385,FDX,Financial news,UPDATE 1 EU probes new China postal law over trade concerns,EU analysing if new China postal law breaches trade rules\n Law favours monopoly China Post excludes foreign firms\n EU business lobby postal industry criticise law\n\n Adds background details \n By Darren Ennis\n BRUSSELS April 29 Reuters The European Commission said\non Wednesday it was investigating whether China broke world\ntrade rules with a new postal law that allows only a state\nmonopoly to deliver domestic letters and documents \n The probe by the European Union exe...,2009-04-29
444,JPM,Financial news,FINRA fines JPMorgan 1 25 million for failures in employee background checks,NEW YORK Reuters The Financial Industry Regulatory Authority said on Tuesday it fined JPMorgan Chase Co s N JPM securities division 1 25 million for failing to conduct adequate background checks on 8 600 employees including four people whose criminal convictions automatically disqualified them from working there Of the four individuals that FINRA found who were subject to statutory disqualification because of some criminal history it said that one worked at JPMorgan for 10 year...,2017-11-21
496,NFLX,Financial news,Tech giants prep for legal fight against FCC s rule repeal,A day after the release of official text of the FCC s order to dismantle net neutrality regulations heavy tech hitters are lining up to hit the courts in opposition Lawsuits can t come until the new order is published in the Federal Register But the Internet Association repping companies including Facebook NASDAQ FB Twitter NYSE TWTR Google GOOG GOOGL Amazon com NASDAQ AMZN and Netflix NASDAQ NFLX says it s joining the fight against the FCC and Chairman Ajit Pai The g...,2018-01-05
838,GOOGL,Financial news,Fair use matters in dancing toddler copyright case U S court,By Jonathan Stempel Reuters Copyright holders must consider fair use before demanding the removal of videos that people post online including on Google Inc s O GOOGL YouTube a U S appeals court ruled on Monday In a closely followed case over a home video of a toddler dancing to the Prince hit Let s Go Crazy the 9th U S Circuit Court of Appeals in San Francisco made it tougher for content providers such as Vivendi SA s PA VIV Universal Music Group to force Internet service...,2015-09-14
850,UPS,Financial news,UPS to pay 8 4 million to resolve U S overcharging probe,WASHINGTON Reuters The U S Justice Department said United Parcel Service Inc N UPS has agreed to pay the United States 8 4 million to resolve allegations it overcharged federal agencies for package delivery services \nThe settlement resolves allegations that from 2007 to 2014 UPS failed to follow its U S contract that required UPS to provide agencies with certain lower

In [210]:
legal_dataset.shape

(7356, 5)

In [211]:
legal_dataset.to_csv('legal_dataset.csv', sep=',', index=False)

In [212]:
investments_dataset = financial_news_dataset.loc[predictions=='M&A | Investments',:]
investments_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
233,IX,Financial news,UPDATE 5 Monex buys Orix unit to form No 2 Japan online broker,Deal priced at estimated 246 million\n Aimed at cutting systems other costs\n Orix to replace Citi as top stockholder of Monex\n Monex shares drop 4 6 pct Orix falls 2 5 pct\n Updates value based on closing share price \n By Yumiko Nishitani and Nathan Layne\n TOKYO Oct 28 Reuters Monex Group Inc Japan s No 3\nonline broker will buy the securities unit of Orix Corp for\nabout 246 million in stock in a deal that will create the\ncountry s second largest online broker \n ...,2009-10-27
510,NBL,Financial news,Devon s Divestiture Goal Reached With Access Pipeline Sale,Devon Energy Corporation NYSE DVN announced that it has entered into a definitive agreement with Wolf Midstream Inc to sell the 50 ownership interest it has in Access Pipeline The deal is valued at 1 1 billion CAD 1 4 billion With the company in talks with prospective buyers to sell interest in the midstream assets this announcement came as no surprise to the business community Details of the DealApart from the 50 stake in the Access Pipeline the sale agreement can potentially...,2016-07-14
565,IP,Financial news,International Paper To Buy Weyerhaeuser s Pulp Business,Paper and packaging firm International Paper Company NYSE IP recently inked a definite agreement with Weyerhaeuser Co NYSE WY one of the world s largest private owners of timberlands to acquire the latter s pulp business for 2 2 billion in cash The transaction is subject to mandatory closing conditions and regulatory approvals and is expected to be completed by fourth quarter 2016 The acquired asset portfolio includes five pulp mills and two converting facilities that produce fl...,2016-05-03
586,VZ,Financial news,Verizon Looking To Cut 1 Billion Off Its Yahoo Bid,On Friday it was reported that telecommunications giant Verizon NYSE VZ is looking to cut 1 billion off its 4 8 billion offer for Yahoo s NASDAQ YHOO core business The New York Post the story \nThis is most likely an after effect of bad news for the web stalwart over the past few weeks Yahoo recently revealed that in 2014 it had been hacked and 500 million accounts including usernames and passwords were stolen \nAnd earlier this week reports surfaced that claimed Yahoo scann...,2016-10-07
769,GS,Financial news,RBS Might Transfer Assets To Netherlands In No Deal Brexit,The Royal Bank of Scotland LON RBS NYSE RBS is planning to shift assets to the Netherlands in March 2019 in case of a no deal Brexit The bank has put forth request for transfer of assets worth about 6 billion and 7 billion of liabilities to its Dutch subsidiary NatWest Markets N V \nThe move is expected to impact nearly 30 of customers in NatWest markets who are located in the European Economic Area \nIn case a deal is struck between Britain and European Union EU the trans...,2018-12-10
864,BDX,Financial news,Cigna To Take Over Express Scripts ESRX For 67 Billion,Integrating medical care and pharmacy benefits under one roof Express Scripts Holding Company NASDAQ ESRX recently announced that it is getting acquired by Cigna Corporation NYSE CI a global health insurance company The acquisition is expected to be completed by Dec 31 2018 Post the acquisition the combined company will invest 200 million in its charitable foundation to support the communities in which it operates Per the definitive agreement Cigna will take over Express Scri...,2018-03-11
1326,BA,Financial news,Embraer s biggest passenger jet may be its best seller,By Brad Haynes and Tim Hepher CANCUN Mexico Reuters The biggest passenger jet ever made by Brazil s Embraer SA SA EMBR3 may become its top selling model emerging from the shadows of smaller siblings due to extra rows and big wings that have changed its economics a senior executive told Reuters The first generation of the 116 seat E195 was outsold more than three to one by the 100 seat E190 but a larger re engined version enteri

In [213]:
investments_dataset.shape


(8724, 5)

In [214]:
investments_dataset.to_csv('investments_dataset.csv', sep=',', index=False)

In [215]:
macro_dataset = financial_news_dataset.loc[predictions=='Macro',:]
macro_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
25,MS,Financial news,11 Reasons Why U S Economic Growth Is The Worst That It Has Been,Those that were predicting that the U S economy would be flying high by now have been proven wrong U S GDP grew at the worst rate in three years during the first quarter of 2017 and many are wondering if this is the beginning of a major economic slowdown Of course when we are dealing with the official numbers that the federal government puts out it is important to acknowledge that they are highly manipulated \nThere are many that have correctly pointed out to me that if the numbers wer...,2017-04-30
92,DE,Financial news,U S farmers tighten belts to compete with cheap LatAm grain,By Mark Weinraub and P J Huffstutter\nCHICAGO Reuters When Kansas farmer Tom Giessel drove over a deer carcass and punctured a tire on his combine during harvest this fall he did not have the time or cash to fix it He borrowed his neighbor s tractor to finish \nU S farmers are cutting costs any way they can to compete against cheaper producers in Argentina and Brazil Four years of global oversupply have pushed down grain prices reduced agricultural revenues and put more expensive p...,2017-10-26
110,C,Financial news,LEI Index Points To Improving Economy But Is It Reliable,Economic numbers out of the US continue to surprise to the downside Just today the jobless claims came in higher than expected The Philadelphia Fed activity came in lower than expected Existing home sales were down 2 6 month over month versus up 0 7 per economists survey The Citigroup US Economic Surprise Index is continuing to trend down It was therefore strange to see the Conference Board US Leading Economic Indicator LEI beating expectations to the upside Does it mean that LEI i...,2012-04-20
323,NWSA,Financial news,World Competitiveness Rankings Europeans Lost Ground,As calls grow for Europe to ease up on its austerity drive a report on global competitiveness argued that cost cutting on the continent has halted growth and antagonized the population creating a number of non competitive economies The World Competitiveness Rankings compiled by the IMD business school in Lausanne identified the U S Switzerland Hong Kong and Sweden as the top four countries while European nations have lost ground not just over the last year but over the last 16 yea...,2013-05-30
361,BA,Financial news,U S Russia suspend nuclear treaty,The U S and Russia suspended a 1987 nuclear weapons treaty on Saturday which banned land based missiles with a range between 300 and 3 400 miles and kept nuclear tipped cruise missiles off the European continent for three decades Arms race Putin said Moscow would follow Washington in pursuing research and development of intermediate range missiles but said he wouldn t deploy them in Europe or elsewhere unless the U S did so Related tickers BA LMT NOC RTN GD BW HON ACM JEC SAI...,2019-02-03
395,NWSA,Financial news,Majors Daily Forecast 27 June 2013,EUR USDTrading range 1 3050 1 2975Trend NeutralSell at 1 3038 SL 1 3070 TP 1 2988USD JPYTrading range 97 55 98 25Trend NeutralBuy at 97 67 SL 97 35 TP 98 12GBP USDTrading range 1 5355 1 5275Trend NeutralSell at 1 5344 SL 1 5376 TP 1 5289USD CHFTrading range 0 9395 0 9465Trend NeutralBuy at 0 9405 SL 0 9373 TP 0 9455The current support resistance levels are EUR USD 1 2978 1 2959 1 2928 1 3098 1 3117 1 3148USD JPY 97 122 96 950 96 672 98 218 98 390 98 668GBP USD 1 5...,2013-06-27
743,ALL,Financial news,In insurance Big Data could lower rates for optimistic tweeters,By Brenna Hughes Neghaiwi ZURICH Reuters When people take to Twitter to comment on the great evening they enjoyed with good food and wonderful friends reducing their monthly insurance bill is probably the last thing on their mind But such tweets could help insurers to price premiums for individuals with research suggesting a direct link between positive posts and a reduced risk of heart disease This could lead to future insurance cover b

In [216]:
macro_dataset.shape

(11210, 5)

In [217]:
macro_dataset.to_csv('macro_dataset.csv', sep=',', index=False)

In [218]:
markets_dataset = financial_news_dataset.loc[predictions=='Markets',:]
markets_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
54,NWSA,Financial news,Street Just Yawns At Europe s Latest Crisis,Goldman Sachs called the Cyprus bank bailout a big deal according to someone who posted in the Rick s Picks chat room yesterday morning Whether true or not as far as Wall Street is concerned the bailout and the punitive measures it will bring against depositors barely even merited a yawn The Dow Industrials fell a meaningless 62 points yesterday in response to the news The close was up sharply from overnight lows that equated to a 180 point drop in the blue chip average We were braced ...,2013-03-19
113,GPAGF,Financial news,Mexico stocks higher at close of trade IPC up 1 00,Investing com Mexico stocks were higher after the close on Monday as gains in the Consumer Staples Industrials and Telecoms Services sectors led shares higher \nAt the close in Mexico the IPC gained 1 00 \nThe best performers of the session on the IPC were Grupo Carso S A B De C V MX GCARSOA1 which rose 5 86 or 4 210 points to trade at 75 930 at the close Meanwhile Alpek S A B De C V MX ALPEKA added 4 18 or 0 840 points to end at 21 000 and Gruma S A B De C V MX GRUM...,2015-08-31
114,TGT,Financial news,FTSE up 0 9 percent early financials gain,Banks gain offsetting weakness in miners\n Schroders Aviva boosted by broker upgrades\n BG Group falls after posting Q2 profit drop\n \n By Harpreet Bhal \n LONDON July 29 Reuters Britain s FTSE 100 share index\nrose in early trade on Wednesday a day after registering the\nfirst decline in the last 12 sessions with a rebound in\nfinancials offsetting weakness in gas firm BG and mining stocks \n By 0815 GMT the FTSE 100 was up 0 9 percent or 39 01 points\nat 4 567 85 points in ...,2009-07-29
141,AMZN,Financial news,Markets Mixed On Last Week s Finish,US equities finished a choppy session and week mixed amid some disappointing economic data and continued pressure from technology stocks Treasuries were higher following an economic calendar that showed housing starts and building permits surprisingly fell and a preliminary read on consumer sentiment cooled Meanwhile crude oil prices were modestly higher as was gold and the US dollar lost ground In equity news Amazon com NASDAQ AMZN inked a deal to acquire Whole Foods Market NASDAQ ...,2017-06-18
215,GS,Financial news,U S stocks mixed after choppy session Dow down 0 15,Investing com U S stocks finished Thursday mixed as investors applauded upbeat U S data and earnings though concerns economies elsewhere may be cooling allowed for choppy trading \nAt the close of U S trading the Dow 30 fell 0 15 the S P 500 index rose 0 01 while the NASDAQ Composite index rose 0 03 \nThe Volatility S P 500 index which measures the outlook for market volatility was down 2 44 at 25 61 \nUpbeat U S data released earlier Thursday drew applause on Wall Street \nTh...,2014-10-16
300,GPAGF,Financial news,Mexico stocks higher at close of trade S P BMV IPC up 0 97,Investing com Mexico stocks were higher after the close on Wednesday as gains in the Healthcare Materials and Financial Services sectors led shares higher \nAt the close in Mexico the S P BMV IPC rose 0 97 to hit a new 3 months high \nThe best performers of the session on the S P BMV IPC were Megacable Holdings S A B De C V MX MEGACPO which rose 3 61 or 3 010 points to trade at 86 380 at the close Meanwhile Grupo Financiero Banorte MX GFNORTEO added 3 02 or 3 420 points t...,2018-01-24
320,CMG,Financial news,Stock Market News For Oct 26 2017,Markets tanked on Wednesday after the Dow and the S P 500 suffered their worst single day declines in almost seven weeks The Dow declined after shares of Boeing nosedived following a steep rise in its production cost The S P 500 ended in the red after a decline in shares of Advanced Micro Devices and Chipotle Mexican Grill weighed heavily on the benchmark Meanwhile new home sales and durable orders for September came in above the consensus estimate The Dow Jones Industrial Average DJIA 

In [219]:
markets_dataset.shape

(24264, 5)

In [220]:
markets_dataset.to_csv('markets_dataset.csv', sep=',', index=False)

In [221]:
politics_dataset = financial_news_dataset.loc[predictions=='Politics',:]
politics_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
761,AAPL,Financial news,Crowd chants Modi Modi as India PM winds up Silicon Valley tour,By Yasmeen Abutaleb SAN JOSE California Reuters Lights flashed and chants of Modi Modi filled the 18 000 seat sports arena in San Jose California on Sunday as the Indian premier took the stage for the final event of his Silicon Valley tour marked by sporadic protests over his human rights record Indian Prime Minister Narendra Modi was winding up a whirlwind two day U S West Coast trip and Sunday s event followed visits to some of the world s biggest technology companies hoping ...,2015-09-28
1620,AAPL,Financial news,Who Benefits From Trump s China Tariffs,Today we have another Trump success story to report To the detriment of China and the US Vietnam benefits \nTariff costs prompt executives to shift production to other countries Once you move you don t go back is the often head claim \nAs a result of Trump s tariffs Manufacturers Move Supply Chains Out of China but who is the beneficiary \nCompanies that make Crocs shoes Yeti beer coolers Roomba vacuums and GoPro cameras are producing goods in other countries to avoid U S tari...,2019-07-15
1678,AGN,Financial news,President Trump at it again with comments on out of control drug prices,Drug and biotech stocks are again the target of disparaging comments from President Trump who says the companies are getting away with murder with their out of control drug prices He says U S consumers should be paying what patients in other countries are paying also meaning that other countries should pay more Selected tickers IBB 0 1 XBI 0 2 PJP IHE 0 1 AMGN 0 7 BIIB 1 2 GILD 1 4 CELG BMY 2 3 MRK 0 1 PFE 0 7 LLY 0 7 NVS 0 3 OTCQX RHHBY 0 4 G...,2017-10-16
2828,AMZN,Financial news,Factbox India s Modi crafts populist measures ahead of elections,By Krishna N Das NEW DELHI Reuters India is weighing measures to woo farmers small business owners and those who are less well off after setbacks in state elections for Prime Minister Narendra Modi s ruling party now readying for a general election due by May Two opinion polls on Thursday suggested that a coalition led by Modi s Hindu nationalist Bharatiya Janata Party BJP could emerge as the largest group in the election while falling short of a majority That could push Modi to...,2019-01-25
3052,GOOGL,Financial news,IForex Daily Looking Ahead To FOMC Speech,The dollar turned broadly higher against the euro and a basket of other major currencies on Tuesday after upbeat U S consumer price inflation and new home sales data fuelled fresh optimism over the strength of the economy The uptick in underlying inflation indicated that the Fed would still have leeway to tighten monetary policy even with inflation running below target Today investors will focus on German Ifo Business Climate and BBA Mortgage Approvals released in the eurozone while o...,2015-03-25
3761,LMT,Financial news,U S senators introduce bill to stop transfer of F 35 fighters to Turkey,WASHINGTON Reuters Four U S senators on Thursday introduced a bipartisan bill to prohibit the transfer of F 35 fighter aircraft to Turkey until the U S government certifies that Ankara will not take delivery of a Russian S 400 air defense system a statement on the move said Turkey is a production partner in the trillion dollar F 35 fighter jet program but Ankara also wants to purchase a Russian missile defense system which the United States says would compromise the security of F 35...,2019-03-28
3868,ICE,Financial news,Democrat Sanders vows to halt immigration raids deportations if elected president,By Simon Lewis\nWASHINGTON Reuters U S Democratic presidential contender Bernie Sanders said on Thursday he would put a moratorium on deportations from the United States and end raids by Immigration and Customs Enforcement ICE NYSE ICE on his first day in office \nSanders said in a statement he would overturn Republican President Donald Trump s border policies if elected and crea

In [222]:
politics_dataset.shape

(4619, 5)

In [223]:
politics_dataset.to_csv('politics_dataset.csv', sep=',', index=False)

In [224]:
personnel_change_dataset = financial_news_dataset.loc[predictions=='Personnel Change',:]
personnel_change_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
111,CSCO,Financial news,BlackBerry names Cisco veteran as global head of sales,By Euan Rocha TORONTO Reuters BlackBerry Ltd named Carl Wiese as head of global sales on Monday an appointment that comes at a critical juncture for the former smartphone leader which needs to spur top line growth for its turnaround efforts to succeed Wiese has spent the past 12 years at Cisco Systems Inc NASDAQ CSCO initially heading advanced technology sales and later Cisco s collaboration product sales efforts At BlackBerry he replaces John Sims as global head of sales Black...,2015-07-13
240,KO,Financial news,Coca Cola names Quincey president fuelling succession speculation,By Anjali Athavaley Reuters Coca Cola Co N KO on Thursday named James Quincey president and chief operating officer setting up the long time company veteran to possibly one day take the helm of the world s largest soda maker The company also said Ahmet Bozer another Coke veteran who had been widely seen as a possible successor to Chief Executive Officer Muhtar Kent will retire Bozer executive vice president and president of Coca Cola International will stay on as an adviser un...,2015-08-13
1197,AAPL,Financial news,Qualcomm to depose Apple services chief Eddy Cue,By Stephen Nellis Reuters Qualcomm NASDAQ QCOM Inc can depose Apple NASDAQ AAPL Inc s services chief Eddy Cue in addition to Chief Executive Tim Cook a magistrate judge in the U S District Court for the Southern District of California ruled on Friday part of the chipmaker s effort to determine whether Apple worked with Samsung to focus regulatory scrutiny on Qualcomm The access to Cue is important because Qualcomm alleges talks between executives at Apple and its rival Samsung El...,2018-05-04
1456,GOOGL,Financial news,Google Canada head exits company names interim head,TORONTO Reuters Alphabet NASDAQ GOOGL Inc said on Tuesday that the head of its Google Canada operations Sam Sebastian has left the company \nSebastian had headed Canadian operations since 2014 overseeing the expansion of its engineering hub in Kitchener Ontario and helping build Google s artificial intelligence technology prowess in the country \nPelmorex Corp the owner of The Weather Network and which was formerly known as Pelmorex Media Inc separately on Tuesday said Sebastian...,2017-07-25
1540,BAC,Financial news,Exclusive Bank of America to hire 50 bankers for Asia dealmaking team in 2020 sources,By Sumeet Chatterjee and Scott Murdoch HONG KONG Reuters Bank of America Corp N BAC will recruit about 50 bankers for its investment banking team in Asia this year as it pushes to bolster its dealmaking business and shore up revenues from the unit three people with direct knowledge of the matter said Bank of America s BofA s Asia headcount expansion comes against the backdrop of a subdued recruitment outlook for the broader investment banking business as global lenders grapple wit...,2020-01-22
2512,KBH,Financial news,KB Home cuts CEO s bonus following tirade at Kathy Griffin,KB Home NYSE KBH is cutting CEO Jeffrey Mezger s annual bonus by 25 following disclosure of a profanity laced tirade at comedian Kathy Griffin a neighbor in Los Angeles Bel Air neighborhood On a two minute recording published by HuffPost the CEO is heard shouting a string of sexist and anti gay obscenities If in the future there is any similar incident Mezger will be dismissed says KBH s board Now read,2017-09-21
3143,PG,Financial news,Procter Gamble Names David S Taylor Chairman Of Board,The Procter Gamble Company NYSE PG recently announced the appointment of David S Taylor as Chairman of the Board effective Jul 1 2016 Taylor will also assume the role of President and CEO Taylor has been with P G since 1980 and played an important role in the creation of its core businesses including Baby Care Family Care Hair Care and Home Care Taylor has served as Group President of Global Beauty Grooming and Health Care Taylor has also led the Fam

In [225]:
personnel_change_dataset.shape

(2690, 5)

In [226]:
personnel_change_dataset.to_csv('personnel_change_dataset.csv', sep=',', index=False)

In [227]:
stock_commentary_dataset = financial_news_dataset.loc[predictions=='Stock Commentary',:]
stock_commentary_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
34,CATM,Financial news,Here s Why You Should Hold Alliance Data ADS Stock Now,Alliance Data Systems Corporation NYSE ADS remains well poised for growth driven by solid performance across its segments sturdy balance sheet and effective capital management The Zacks Rank 3 Hold private label credit card processing company has been efficiently utilizing its shareholders funds as reflected through its return on equity of 58 9 which betters the average of 42 6 The stock carries an impressive of A This helps to identify stocks with the most attractive value ...,2018-12-17
41,LHX,Financial news,What Makes Harris HRS A Strong Momentum Stock Buy Now,Momentum investing is all about the idea of following a stock s recent trend which can be in either direction In the long context investors will essentially be buying high but hoping to sell even higher And for investors following this methodology taking advantage of trends in a stock s price is key once a stock establishes a course it is more than likely to continue moving in that direction The goal is that once a stock heads down a fixed path it will lead to timely and profit...,2019-02-12
88,PFG,Financial news,Are You Looking For A High Growth Dividend Stock Principal Financial PFG Could Be A Great Choice,Whether it s through stocks bonds ETFs or other types of securities all investors love seeing their portfolios score big returns But when you re an income investor your primary focus is generating consistent cash flow from each of your liquid investments \nWhile cash flow can come from bond interest or interest from other types of investments income investors hone in on dividends A dividend is that coveted distribution of a company s earnings paid out to shareholders and investors o...,2019-06-19
125,JBLU,Financial news,Here s Where I ll Buy JetBlue,JetBlue Airways NASDAQ JBLU has been steadily declining since July 13 2017 At that time the stock peaked out at 24 13 a share Today JBLU is trading at 20 27 a share Traders and investors should note that JetBlue bottomed on the weekly chart in February 2017 at 18 71 This tells us that the stock will have very good chart support around this level if it is retested I ll be looking to pick up JBLU around the low 19 level with a weekly chart stop loss below 17,2017-08-21
155,LAZ,Financial news,Why Regions Financial RF Is A Great Pick Right Now,Shares of Birmingham based banking giant Regions Financial Corporation NYSE RF climbed more than 12 so far this year on the NYSE The company continues to reflect strength in several areas amid the current challenging environment with low rates and regulatory pressure Why the Surge May ContinueDiversified fee income drives revenue growth Regions Financial remains committed toward diversifying its revenue streams During third quarter 2016 the company recorded 12 year over year incre...,2016-11-02
214,TXN,Financial news,Texas Instruments A Growth Stock Paying A 3 2 Dividend Yield,Traditionally the tech sector has not been a source of dividend stocks Many tech companies particularly those at earlier stages of their development do not pay dividends at all Instead they typically need to reinvest as much cash flow as possible back into growing the business And even among tech companies that do pay dividends to shareholders yields are usually low \nHowever in rare cases investors can find highly profitable mature tech companies with enough cash flow to pay high ...,2018-11-20
463,NIO,Financial news,Three Foreign Stocks To Watch STNE QTT NIO,StoneCo Qutoutiao and NIO are three foreign stocks that high relative strength small float and belong to an industry that is hot now \nStoneCo Ltd STNE \nStoneCo Ltd NASDAQ STNE is the Square NYSE SQ of Brazil It recovered from a 50 slump in December and consolidating near its all time highs Due to their small float recent IPOs tend to get hit hard during severe market corrections They also tend to be among the strongest performe

In [228]:
stock_commentary_dataset.shape

(24364, 5)

In [229]:
stock_commentary_dataset.to_csv('stock_commentary_dataset.csv', sep=',', index=False)

In [231]:
stock_movement_dataset = financial_news_dataset.loc[predictions=='Stock Movement',:]
stock_movement_dataset.head(10)

,Ticker,Data source,Headline,Content,Release date
7,GE,Financial news,General Electric Shares Jump Despite Lower Than Expected Earnings,Investing com Shares of General Electric NYSE GE rose in pre market trading as its financial results on Wednesday came in lower than expected \nThe conglomerate reported fourth quarter earnings per share of 0 27 on revenue of 31 4 billion Analysts had expected earnings of 0 29 and revenue of 34 06 \nThe company s shares were up 4 45 ahead of the opening bell The firm s stock had declined more than 43 over the past year since Monday s close,2018-01-24
77,GOOGL,Financial news,Bitcoin rockets above 5 000 to all time high,By Jemima Kelly LONDON Reuters Bitcoin smashed through the 5 000 barrier for the first time on Thursday jumping as much as 8 percent on the day as investors shrugged off the latest warnings on the risks of buying into the booming cryptocurrency market Bitcoin the biggest and best known cryptocurrency has chalked up a more than fivefold increase in price this year Typically for bitcoin which at less than nine years old is still highly volatile and illiquid compared with tradition...,2017-10-12
162,AXP,Financial news,Visa Or Mastercard Which Stock Looks Better Post Results,Visa Inc NYSE V and Mastercard Inc NYSE MA leaders in the payment processing industry recently reported results for the January March quarter While Mastercard reported first quarter results Visa came up with second quarter fiscal 2019 numbers \nBoth Visa and Mastercard with a market capitalization of 314 billion and 247 billion respectively are leaders in the payment processing industry with their worldwide payment network partnerships with several retailers banks and m...,2019-05-15
211,TM,Financial news,Japanese Car Makers Rally On Weak Yen Hyundai And Kia Misstep,The beleaguered Japanese auto shares are rallying today on the weaker Japanese yen and overstated mileage claims by South Korean rivals Hyundai and Kia The rapid decline of the yen against the U S dollar this week following Japanese prime minister Noda s decision to call a Lower House election for December 16 is the best news any Japanese car company has had in months The yen is trading at an 81 handle for the first time since April which could add a lot to the bottom lines of Toyota TM...,2012-11-18
228,BA,Financial news,Esterline Technologies ESL Soars To 52 Week High Time To Cash Out,Have you been paying attention to shares of Esterline Technologies ESL Shares have been on the move with the stock up 3 5 over the past month The stock hit a new 52 week high of 122 14 in the previous session Esterline Technologies has gained 63 4 since the start of the year compared to the 6 9 move for the Zacks Aerospace sector and the 3 9 return for the Zacks Aerospace Defense Equipment industry \nWhat s Driving the Outperformance \nThe stock has a great record of positive e...,2018-12-27
249,SYY,Financial news,Stocks making the biggest moves midday Tesla Gilead Sciences Nike Uber more,Check out the companies making headlines midday Monday Tesla Shares of Tesla jumped 19 9 surging past the 700 mark for the first time after Argus Research raised its price target to the highest on Wall Street at 808 a share Argus said it sees continued growth in sales of Tesla s Model S and Model X models as well as strong demand for the new Model 3 Nike The shoe giant rose 3 1 after UBS upgraded the stock to buy from neutral and JPMorgan added Nike to its analyst focus list ...,2020-02-03
258,LOW,Financial news,Lowe s Tanks On Earnings Possible Short Term Bottom,Lowe s Companies NYSE LOW fell 8 on Wednesday morning after a mixed earnings report While the company beat on revenue and same store sales it missed by 0 13 on earnings per share EPS \nLOW reported adjusted EPS of 0 74 and total revenue of 15 49 billion compared to the market s expectations of 0 87 and 15 33 billion Same store sales increased by 4 1 compared to estimates of 3 1 \nThe stock recovered to 5 after CFO Marshall Cr

In [232]:
stock_movement_dataset.shape

(19915, 5)

In [233]:
stock_movement_dataset.to_csv('stock_movement_dataset.csv', sep=',', index=False)

In [234]:
twitter_dataset.to_csv('twitter_dataset.csv', sep=',', index=False)